In [ ]:
!pip install --quiet openai python-dotenv

import os
import sys
import openai
from dotenv import load_dotenv
from google.colab import drive

drive.mount('/content/drive')

path_utils ="/content/drive/Othercomputers/My Computer/Files_win10/python/py310/"
path_dotenv = "/content/drive/Othercomputers/My Computer/Files_win10/python/py310/.env/my_api_key.env"

# Directory to save videos
save_dir = '/content/drive/MyDrive/0_YouTube'

In [7]:
# import os
# import sys


# current_dir = os.getcwd()

# # Parent directory where myUtils is located
# path_utils = os.path.dirname(current_dir)

# path_dotenv = "/content/drive/Othercomputers/My Computer/Files_win10/python/py310/.env/my_api_key.env"
# path_dotenv = path_utils + "/.env//my_api_key.env"

# # Save downloads to Desktop/youtube directory
# save_dir = os.path.join(os.environ['USERPROFILE'], 'Desktop\\youtube')

# # Create save_dir if it does not exists
# os.makedirs(save_dir, exist_ok=True)

# print(f'current directory: {current_dir}')
# print(f'path_utils: {path_utils}')
# print(f'path_dotenv: {path_dotenv}')
# print(f'save_dir: {save_dir}')

current directory: c:\Users\ping\Files_win10\python\py310\Youtube
path_utils: c:\Users\ping\Files_win10\python\py310
path_dotenv: c:\Users\ping\Files_win10\python\py310/.env//my_api_key.env
save_dir: C:\Users\ping\Desktop\youtube


In [2]:
sys.path.append(path_utils)
from myUtils import get_file_names, check_file_size, find_strings_with_substring

In [8]:
import openai
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(path_dotenv)

# Access the environment variables
SECRET_KEY1 = os.environ.get("SECRET_KEY")
DATABASE_PASSWORD2 = os.environ.get("DATABASE_PASSWORD")
print(f"SECRET_KEY = {SECRET_KEY1}")
print(f"DATABASE_PASSWORD = {DATABASE_PASSWORD2}")
openai.api_key = os.environ['OPENAI_API_KEY']

SECRET_KEY = my_secret_key
DATABASE_PASSWORD = my_db_pwd


In [ ]:
# Create a directory named 'save_dir' in the current working directory
os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

In [ ]:
# Change current directory to save_dir
os.chdir(save_dir)
print(f'current directory: {os.getcwd()}')

Find mp3 files to transcribe

In [ ]:
# Files in current directory
filenames = get_file_names(save_dir, ext=None)
print(f'files in current directory {os.getcwd()}:')
for filename in filenames:
  print(filename)

In [ ]:
# Get filenames end with txt or mp3
txt_filenames = find_strings_with_substring(filenames, "txt", "end")
mp3_filenames = find_strings_with_substring(filenames, "mp3", "end")
print(f'mp3 filenames: {mp3_filenames}')
print(f'txt filenames: {txt_filenames}')

In [ ]:
# strip .mp3 from mp3_files
mp3_titles = [file.replace('.mp3', '') for file in mp3_filenames]
print(f'mp3_titles: {mp3_titles}')

In [ ]:
# concatenate txt_filenames into a string
txt_filenames_string = ', '.join(txt_filenames)
print(f'txt_filenames_string: {txt_filenames_string}')

In [ ]:
# find mp3_titles that do not have a text file
mp3_titles_without_text_files = []
for mp3_title in mp3_titles:
  if mp3_title in txt_filenames_string:
    pass
  else:
    mp3_titles_without_text_files.append(mp3_title)

print(f'mp3 titles without text files: {mp3_titles_without_text_files}')


Transcribe mp3 audio files, 25 MB Max, to text.  
https://platform.openai.com/docs/guides/speech-to-text/quickstart

In [ ]:
from openai import OpenAI
client = OpenAI()

for mp3_title in mp3_titles_without_text_files:
  txt_filename = mp3_title + '.txt'
  mp3_file = mp3_title + '.mp3'

  file_info = os.stat(mp3_file)
  file_size_bytes = file_info.st_size
  file_size_mb = file_size_bytes / (1024 * 1024)

  print(f'{mp3_file}: {round(file_size_mb, 1)} MB')

  try:
    check_file_size(file_path=mp3_file, file_size_limit_MB=25)

    # Path to your MP3 file
    audio_file = open(mp3_file, "rb")
    transcription = client.audio.transcriptions.create(
      model="whisper-1",
      file=audio_file
    )

    # Save transcription text to file
    with open(txt_filename, 'w') as f:
      f.write(transcription.text)
      print(f'transcribed text saved as: {txt_filename}')

  except ValueError as e:
    print(e)

  print('---')
